# OpenSearch の基本的な検索機能
> この章は、`basic-search.ipynb` を元に作成しています。

## 概要
本ラボでは、OpenSearch Serverless が提供する多様な検索機能について、実際に検索クエリを実行しながら確認していきます。

### ラボの構成
本ラボでは、ノートブック環境（EC2 へ Remote Develop 接続した VSCode）および Amazon OpenSearch Serverless を使用します。

<img src="./img/architecture.png" width="512">

### 使用するデータセット
本ラボでは、Amazon OpenSearch Service [デベロッパーガイド](https://docs.aws.amazon.com/ja_jp/opensearch-service/latest/developerguide/what-is.html)の[チュートリアル](https://docs.aws.amazon.com/ja_jp/opensearch-service/latest/developerguide/search-example.html)内でも使用している[サンプルムービーデータセット](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/samples/sample-movies.zip)を使用します。


## 事前作業

### パッケージインストール
実行する前に、タブの右上のカーネルの選択を確認してください。

> opensearch と awswrangler のバージョンの依存関係のため、opensearch-py 側を止めています。

In [1]:
!uv add "opensearch-py<3" requests-aws4auth awswrangler[opensearch] python-dotenv

Resolved 59 packages in 25ms
Audited 52 packages in 0.01ms


### 環境変数
`.env`ファイルに、以下の環境変数を設定します。

- AOSS_SEARCH_HOST=`AOSS の検索コレクションのエンドポイントのホスト名`
- AOSS_VECTOR_HOST=`AOSS のベクトル検索コレクションのエンドポイントのホスト名`
- AOSS_ROLE_ARN=`AOSS から Bedrock へアクセスするために作成したロールのARN`

設定できたら、以下のコマンドを実行します。

In [2]:
%reload_ext dotenv
%dotenv -o

### インポート

In [3]:
import boto3
import json
import logging
import awswrangler as wr
import pandas as pd
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import os

### 共通変数のセット

In [4]:
default_region = boto3.Session().region_name
logging.getLogger().setLevel(logging.ERROR)

### OpenSearch Serverless への接続確認
OpenSearch Server のセキュリティ設定により、API リクエストが許可されているかを確認します。

In [5]:
aoss_host = os.getenv("AOSS_SEARCH_HOST")

credentials = boto3.Session().get_credentials()
service_code = "aoss"
auth = AWSV4SignerAuth(credentials=credentials, region=default_region, service=service_code)
opensearch_client = OpenSearch(
    hosts=[{"host": aoss_host, "port": 443}],
    http_compress=True, 
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class = RequestsHttpConnection
)
opensearch_client.cat.indices()

' OPEN movies      T_HD9JgB_vf32jNhhv_N   5000 0 2.9mb 2.9mb\n OPEN movie-users W_Hg9JgB_vf32jNhF_8H      1 0 3.6kb 3.6kb\n'

## インデックスの作成・データロード
インデックスを作成し、検索用のサンプルデータを格納します。

### サンプルデータの準備

In [6]:
dataset_dir = "./dataset/sample-movies"

%mkdir -p $dataset_dir

# ファイルダウンロード
!curl -s -o $dataset_dir/sample-movies.zip https://docs.aws.amazon.com/opensearch-service/latest/developerguide/samples/sample-movies.zip

# zip ファイルから sample-movies.bulk のみを展開
!unzip -oq $dataset_dir/sample-movies.zip sample-movies.bulk -d $dataset_dir

# .bulk ファイルから実データ行だけを抜き出して jsonl ファイルとして保存
!grep -v "_index" $dataset_dir/sample-movies.bulk > $dataset_dir/sample-movies.jsonl

### インデックスの作成
本ラボでは、全文検索を意識した [Analyzer][analyzer] や [Normalizer][normalizer] を設定しています。詳細は以降のセクションで順次解説していきます。

[analyzer]: https://opensearch.org/docs/latest/analyzers/supported-analyzers/index/
[normalizer]: https://opensearch.org/docs/latest/analyzers/normalizers/

In [7]:
index_name = "movies"

payload = {
  "settings": {
    "index": {
      "number_of_shards": 1,
      "number_of_replicas": 0
    },
    "analysis": {
      "normalizer": {
        "lowercase_normalizer": {
          "type": "custom",
          "filter": ["lowercase"]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "id":  { "type": "keyword" },
      "directors":  { "type": "text" },
      "release_date": {"type": "date"},
      "rating": {"type": "scaled_float", "scaling_factor": 10},
      "genres":  {
        "type": "keyword",
        "normalizer": "lowercase_normalizer"
      },
      "image_url":  { "type": "keyword" },
      "plot":  {
        "type": "text",
        "analyzer": "english"
      },
      "title":  { "type": "text" },
      "rank": { "type": "integer" },
      "running_time_secs": { "type": "integer" },
      "actors":  { "type": "text" },
      "year": {"type": "short"},
      "type": { "type": "keyword" } 
    }
  }
}

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスの作成を行う
print("# create index")
response = opensearch_client.indices.create(index=index_name, body=payload)
print(json.dumps(response, indent=2))

# delete index
{
  "acknowledged": true
}
# create index
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "movies"
}


### ドキュメントのロード
ドキュメントのロードを行います。ドキュメントのロードは bulk API を使用することで効率よく進められますが、データ処理フレームワークを利用することでより簡単にデータを取り込むことも可能です。本ワークショップでは、[AWS SDK for Pandas][aws-sdk-pandas] を使用したデータ取り込みを行います。

[aws-sdk-pandas]: https://github.com/aws/aws-sdk-pandas

In [8]:
%%time
index_name = "movies"
file_path = f"{dataset_dir}/sample-movies.jsonl"

response = wr.opensearch.index_json(
    client=opensearch_client,
    path=file_path,
    use_threads=True, #クライアントの CPU 数に応じて自動で書き込みを並列化
    id_keys=["id"],
    index=index_name,
    bulk_size=200, # 200 件ずつ書き込み,
    refresh=False # 書き込み処理実行中の refresh (セグメントの書き出し) を無効化
)

CPU times: user 346 ms, sys: 44 ms, total: 390 ms
Wall time: 6.66 s


response["success"] の値が jsonl ファイルの行数と一致しているかを確認します。True が表示される場合は全件登録に成功していると判断できます。

In [9]:
response["success"] == len(open(file_path).readlines())

True

## ドキュメント検索
ドキュメント検索は [Search][search] API を使用します。Search API は様々なパラメーターを持ちます。
基本的なパラメーターはリクエストボディに含めることが可能です。以下は代表的なパラメーターです。

- query: 検索条件を記載します
- aggs: 集計条件を記載します
- sort: ソート条件を記載します
- size: 取得する検索結果の件数を指定します

以降のセクションでは、検索要件に応じた query/aggs/sort の書き方を解説します。

[search]: https://opensearch.org/docs/latest/api-reference/search/

### [全文検索][full-text]
全文検索は OpenSearch の根幹をなす機能です。OpenSearch では文字列をトークンと呼ばれる単位に分割してインデックスに格納することで、高速な全文検索(部分一致検索)を実現しています。

[full-text]: https://opensearch.org/docs/latest/query-dsl/full-text/index/

#### 逐次検索と転置インデックス
逐次検索とは、検索対象の複数のデータを順次照合することで対象となるデータを抽出する方法です。事前にインデックスを作成せずに検索を行うことができますが、順次データの走査を行うためデータサイズに応じて検索時間が伸びていきます。UNIX の grep コマンドは逐次検索にあたります。

<img src="./img/grep.png" width=1024>


一方、OpenSearch は全文検索向けに、Apache Lucene が提供する転置インデックスを使用しています。転置インデックスは特定の見出し語に対応する文書 ID と出現位置の情報を持っています。これにより、特定語句にマッチする検索結果を素早く返すことを可能としています。

<img src="./img/inverted_index.png" width=1024>


#### text 型とアナライザー
全文検索の対象となるデータは、text 型のフィールドに格納される必要があります。text 型のフィールドに格納されたデータは、以下の流れで処理され、転置インデックスに登録されます

1. Character Filter によるテキスト内の文字の置換、削除 (オプション)
2. Tokenizer によるテキストのトークン分割 (必須)
3. Token Filter によるトークンの正規化処理 (オプション)

これらの処理を通じて作成されたトークンがインデックスに格納されることで、キーワードによる文字列の検索が可能となります。この一連の処理をアナライズといい、これらのコンポーネントの集合体をアナライザーと呼びます。

<img src="./img/analyzer.png" width=1024>

アナライザーは、インデックスにドキュメントを格納する際の文字列の処理、また検索時の検索キーワードの処理に用いられます。一般的には格納と検索に同じアナライザーを用いますが、オートコンプリートなど一部のユースケースでは格納時と検索時で異なるアナライザーを使用します。

アナライザーは text 型フィールドの新規定義時にのみ指定が可能です。text 型フィールドに対してアナライザーを明示的に指定しない場合、OpenSearch はデフォルトで standard アナライザーを設定します。フィールドに設定されたアナライザーを後から変更することはできないため、事前に検索要件を確認した上でカスタムアナライザーを定義することがより良い検索体験の提供に繋がります。

OpenSearch では、いくつかの言語向けにアナライザーが標準で提供されています。これらのアナライザーは、予め決められた Character Filter, Tokenizer, Token Filter で構成されています。

デフォルトのアナライザーである Standard Analyzer は以下のコンポーネントで構成されており、主に英語などの半角スペースでターム(語句)が区切られている言語の文章解析に使用できます。トークンは Lower Case Token Filter によりすべて小文字に変換され格納されるため、大文字・小文字の違いに影響されない検索が可能です。

- Character Filter: なし
- Tokenizer: Standard Tokenizer
- Token Filter: Lower Case Token Filter

OpenSearch では英語やフランス語、日本語など各言語に応じたアナライザーのプリセットを提供しています。本ワークショップでは plot フィールドに english アナライザーをセットすることで、解説文による部分検索の精度を高めています。


<div class="alert alert-block alert-info"> 
<b>Tips: カスタムアナライザ</b>

標準で提供されているアナライザーが要件を満たさない場合は、任意の Character Filter, Tokenizer, Token Filter を組み合わせたカスタムアナライザーを使用できます。

例えば、standard analyzer に加えて以下の処理を追加したい場合は、カスタムアナライザーを定義します。

トークンのステミング(sampling -> sample や cars -> car といった語幹の統一)を行う
a や the などのストップワードを除去する。ストップワード判定の際、大文字小文字の違いは無視する
カスタムアナライザーは、インデックス作成時の anaysis オプション配下で定義可能です。カスタムアナライザーをテストする場合は、カスタムアナライザーが定義されているインデックスに対して　_analyze API を発行します。

standard アナライザーに対する _analyze API 結果と比較してみると、on などの不要なトークンが削除され、意味のある単語のみが抽出されていることが分かります。また一部の単語についてはステミングされていることも確認できます。


</div>

#### アナライザーの動作確認
_analyze API を使用することで、アナライザーの動作確認を行えます。standard と english など、アナライザー毎の処理結果の違いを見ることにも役立ちます


In [10]:
payload = {
  "text": "OpenSearch is a distributed search and analytics engine based on Apache Lucene.",
  "analyzer": "standard"
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_standard = pd.json_normalize(response["tokens"])
print(json.dumps(response, indent=2))

{
  "tokens": [
    {
      "token": "opensearch",
      "start_offset": 0,
      "end_offset": 10,
      "type": "<ALPHANUM>",
      "position": 0
    },
    {
      "token": "is",
      "start_offset": 11,
      "end_offset": 13,
      "type": "<ALPHANUM>",
      "position": 1
    },
    {
      "token": "a",
      "start_offset": 14,
      "end_offset": 15,
      "type": "<ALPHANUM>",
      "position": 2
    },
    {
      "token": "distributed",
      "start_offset": 16,
      "end_offset": 27,
      "type": "<ALPHANUM>",
      "position": 3
    },
    {
      "token": "search",
      "start_offset": 28,
      "end_offset": 34,
      "type": "<ALPHANUM>",
      "position": 4
    },
    {
      "token": "and",
      "start_offset": 35,
      "end_offset": 38,
      "type": "<ALPHANUM>",
      "position": 5
    },
    {
      "token": "analytics",
      "start_offset": 39,
      "end_offset": 48,
      "type": "<ALPHANUM>",
      "position": 6
    },
    {
      "token": "engine",
  

In [11]:
payload = {
  "text": "OpenSearch is a distributed search and analytics engine based on Apache Lucene.",
  "analyzer": "english"
}

response = opensearch_client.indices.analyze(
  body=payload
)

df_english = pd.json_normalize(response["tokens"])
print(json.dumps(response, indent=2))

{
  "tokens": [
    {
      "token": "opensearch",
      "start_offset": 0,
      "end_offset": 10,
      "type": "<ALPHANUM>",
      "position": 0
    },
    {
      "token": "distribut",
      "start_offset": 16,
      "end_offset": 27,
      "type": "<ALPHANUM>",
      "position": 3
    },
    {
      "token": "search",
      "start_offset": 28,
      "end_offset": 34,
      "type": "<ALPHANUM>",
      "position": 4
    },
    {
      "token": "analyt",
      "start_offset": 39,
      "end_offset": 48,
      "type": "<ALPHANUM>",
      "position": 6
    },
    {
      "token": "engin",
      "start_offset": 49,
      "end_offset": 55,
      "type": "<ALPHANUM>",
      "position": 7
    },
    {
      "token": "base",
      "start_offset": 56,
      "end_offset": 61,
      "type": "<ALPHANUM>",
      "position": 8
    },
    {
      "token": "apach",
      "start_offset": 65,
      "end_offset": 71,
      "type": "<ALPHANUM>",
      "position": 10
    },
    {
      "token": "lucen",

各アナライザーによって出力されるトークンを表形式で比較してみます。

standard analyzer はシンプルに文章の単語分割が行われています。一方、 english analyzer では is や a といった検索上意味を持たないワード(ストップワード)の除去やステミング(distrubuted -> distribut など、単語の変化しない前方部分のみの抽出)なども行われています。

english analyzer の方が一見すると検索精度が良いようにみえますが、"To be, or not to be, that is the question" を english analyzer で処理すると "question" しか残らないなど、対象のテキストによっては思わぬ副作用を生みます。

例えば映画のタイトル検索では、"avengers" と "avenger" は区別したいという要件がある場合、english analyzer ではなく standard analyzer を使用することが望ましいと考えられます。一方で映画のプロット検索ではそこまで厳密な区別が要求されないと考えられること、ストップワードを除去した方がノイズが少ないと考えられることから、english analyzer を使用することが望ましいと考えられます。

以上の点を踏まえて、本ラボでは、title フィールドについては standard analyzer を、plot フィールドについては english analyzer をセットしていきます。

In [12]:
pd.merge(df_standard, df_english, on=["start_offset","end_offset", "position", "type"], how="left").rename(columns={"token_x": "token_standard", "token_y": "token_english"}).reindex(["start_offset","end_offset","position","type","token_standard","token_english"],axis=1).fillna("")

,start_offset,end_offset,position,type,token_standard,token_english
0,0,10,0,<ALPHANUM>,opensearch,opensearch
1,11,13,1,<ALPHANUM>,is,
2,14,15,2,<ALPHANUM>,a,
3,16,27,3,<ALPHANUM>,distributed,distribut
4,28,34,4,<ALPHANUM>,search,search
5,35,38,5,<ALPHANUM>,and,
6,39,48,6,<ALPHANUM>,analytics,analyt
7,49,55,7,<ALPHANUM>,engine,engin
8,56,61,8,<ALPHANUM>,based,base
9,62,64,9,<ALPHANUM>,on,


#### match query による全文検索
match query は、単一フィールドに対する全文検索を実行するものです。以下は "avengers" を含むタイトルの映画を検索するクエリです。

In [13]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "title": "avengers"
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

print(json.dumps(response, indent=2))

{
  "took": 99,
  "timed_out": false,
  "_shards": {
    "total": 0,
    "successful": 0,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 3,
      "relation": "eq"
    },
    "max_score": 6.783779,
    "hits": [
      {
        "_index": "movies",
        "_id": "tt0848228",
        "_score": 6.783779,
        "_source": {
          "directors": [
            "Joss Whedon"
          ],
          "release_date": "2012-04-11T00:00:00Z",
          "rating": 8.2,
          "genres": [
            "Action",
            "Fantasy"
          ],
          "image_url": "https://m.media-amazon.com/images/M/MV5BMTk2NTI1MTU4N15BMl5BanBnXkFtZTcwODg0OTY0Nw@@._V1_SX400_.jpg",
          "plot": "Nick Fury of S.H.I.E.L.D. assembles a team of superhumans to save the planet from Loki and his army.",
          "title": "The Avengers",
          "rank": 48,
          "running_time_secs": 8580,
          "actors": [
            "Robert Downey Jr.",
            "Chris Evans",


OpenSearch の検索 API の実行結果は JSON 形式で返却されます。これを表形式に変換、出力した結果は以下の通りです。
本ワークショップでは、以降見やすさを重視して結果を表形式で出力します。`pd.json_normalize` から始まる行をコメントアウトし、コメントアウトされている `print` から始まる行のコメントを解除することで JSON による出力結果を確認することもできます

```
# print(json.dumps(response, indent=2))
pd.json_normalize(response["hits"]["hits"])
```

In [14]:
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0848228,6.783779,[Joss Whedon],2012-04-11T00:00:00Z,8.2,"[Action, Fantasy]",https://m.media-amazon.com/images/M/MV5BMTk2NT...,Nick Fury of S.H.I.E.L.D. assembles a team of ...,The Avengers,48,8580.0,"[Robert Downey Jr., Chris Evans, Scarlett Joha...",2012,tt0848228,add
1,movies,tt0118661,6.751337,[Jeremiah S. Chechik],1998-08-13T00:00:00Z,3.5,"[Action, Adventure, Sci-Fi, Thriller]",https://m.media-amazon.com/images/M/MV5BMjA3Nz...,Two British agents (John Steed and Emma Peel) ...,The Avengers,3276,5340.0,"[Ralph Fiennes, Uma Thurman, Sean Connery]",1998,tt0118661,add
2,movies,tt2395427,4.379196,[Joss Whedon],2015-04-29T00:00:00Z,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BNjA3MT...,NaN,The Avengers: Age of Ultron,40,NaN,"[Scarlett Johansson, Chris Hemsworth, James Sp...",2015,tt2395427,add


title フィールドについては avenger と avengers は分けて扱われるため、avengers ではなく avenger で検索すると異なる結果が得られます

In [15]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "title": "avenger"
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0458339,4.420658,[Joe Johnston],2011-07-19T00:00:00Z,6.8,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTYzOT...,"After being deemed unfit for military service,...",Captain America: The First Avenger,253,7440,"[Chris Evans, Hugo Weaving, Samuel L. Jackson]",2011,tt0458339,add


一方、plot フィールドは english analyzer によりステミングが行われるため、superhero で検索した際に superheros を含むドキュメントもヒットします

[Highlight][highlight] と呼ばれる機能を使うことで、フィールド内のどの文字列(トークン)でヒットしたかを確認することができます。

デフォルトでは、ヒットしたトークンは em タグで囲まれて出力されます。タグは変更が可能です。

[highlight]: https://opensearch.org/docs/latest/search-plugins/searching-data/highlight/

In [16]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "match": {
      "plot": "superhero"
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  } 
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0383060,7.451531,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,Former superhero Jack is called back to work t...,Zoom,1395,5580.0,"[Tim Allen, Courteney Cox, Chevy Chase]",2006.0,tt0383060,add,[Former <em>superhero</em> Jack is called back...
1,movies,tt0132347,7.161061,[Kinka Usher],1999-07-22T00:00:00Z,5.9,"[Action, Comedy, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM2Nj...,A group of inept amateur superheroes must try ...,Mystery Men,2279,7260.0,"[Ben Stiller, Janeane Garofalo, William H. Macy]",1999.0,tt0132347,add,[A group of inept amateur <em>superheroes</em>...
2,movies,tt0451279,6.797963,NaN,NaN,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",NaN,An Amazon princess comes to the world of Man t...,Wonder Woman,2806,NaN,NaN,NaN,tt0451279,add,[An Amazon princess comes to the world of Man ...


#### match query における複数キーワードによる検索 (OR / AND)
テキスト検索を行う際、複数のキーワードによる OR or AND 検索を行う要件があります。match query も複数キーワード検索に対応しています。
複数キーワードを入力した場合、デフォルトでは OR 検索となります。

In [17]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "title": {
        "query": "avenger avengers"
      }
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  } 
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0848228,6.783779,[Joss Whedon],2012-04-11T00:00:00Z,8.2,"[Action, Fantasy]",https://m.media-amazon.com/images/M/MV5BMTk2NT...,Nick Fury of S.H.I.E.L.D. assembles a team of ...,The Avengers,48,8580.0,"[Robert Downey Jr., Chris Evans, Scarlett Joha...",2012,tt0848228,add
1,movies,tt0118661,6.751337,[Jeremiah S. Chechik],1998-08-13T00:00:00Z,3.5,"[Action, Adventure, Sci-Fi, Thriller]",https://m.media-amazon.com/images/M/MV5BMjA3Nz...,Two British agents (John Steed and Emma Peel) ...,The Avengers,3276,5340.0,"[Ralph Fiennes, Uma Thurman, Sean Connery]",1998,tt0118661,add
2,movies,tt0458339,4.420658,[Joe Johnston],2011-07-19T00:00:00Z,6.8,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTYzOT...,"After being deemed unfit for military service,...",Captain America: The First Avenger,253,7440.0,"[Chris Evans, Hugo Weaving, Samuel L. Jackson]",2011,tt0458339,add
3,movies,tt2395427,4.379196,[Joss Whedon],2015-04-29T00:00:00Z,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BNjA3MT...,NaN,The Avengers: Age of Ultron,40,NaN,"[Scarlett Johansson, Chris Hemsworth, James Sp...",2015,tt2395427,add


AND 検索は、operator オプションに明示的に and と指定することで実装可能です。同オプションのデフォルトは or となっています。

In [18]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "plot": {
        "query": "superhero transform academy",
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  } 
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0383060,19.915394,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,Former superhero Jack is called back to work t...,Zoom,1395,5580,"[Tim Allen, Courteney Cox, Chevy Chase]",2006,tt0383060,add,[Former <em>superhero</em> Jack is called back...


[minimum_should_match][minimum_should_match] オプションを使うことで、クエリに含まれるキーワードにおいて、特定数、もしくは特定割合のワード数マッチすればヒットしたとみなすことも可能です。

[minimum_should_match]: https://opensearch.org/docs/latest/query-dsl/minimum-should-match/

In [19]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "plot": {
        "query": "superhero transform academy",
        "minimum_should_match": "2"
      }
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  } 
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0383060,19.915394,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,Former superhero Jack is called back to work t...,Zoom,1395,5580,"[Tim Allen, Courteney Cox, Chevy Chase]",2006,tt0383060,add,[Former <em>superhero</em> Jack is called back...
1,movies,tt0120903,10.692701,[Bryan Singer],2000-07-13T00:00:00Z,7.4,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTYxMT...,Two mutants come to a private academy for thei...,X-Men,471,6240,"[Patrick Stewart, Hugh Jackman, Ian McKellen]",2000,tt0120903,add,[Two mutants come to a private <em>academy</em...
2,movies,tt1512235,8.804738,[James Gunn],2010-09-10T00:00:00Z,6.7,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BMTcxMD...,After his wife falls under the influence of a ...,Super,1449,5760,"[Rainn Wilson, Ellen Page, Liv Tyler]",2010,tt1512235,add,[After his wife falls under the influence of a...


#### multi match query によるフィールド横断の全文検索
同一の検索条件で複数フィールドにまたがった検索を実行したい場合は、[Multi-match][multi-match] クエリを使用します。

title もしくは plot フィールドに wind の文字列を含むドキュメントを検索する場合、Multi-match を利用して以下のように記述することができます。

[multi-match]: https://opensearch.org/docs/latest/query-dsl/full-text/multi-match/

In [20]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "multi_match": {
      "query": "wind",
      "fields": ["title", "plot"]
    }
  },
  "highlight": {
    "fields": {
      "title": {},
      "plot": {}
    }
  } 
}

response = opensearch_client.search(
    index=index_name,
    body=payload,
    filter_path="hits.hits"
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt1045778,6.200220,[Harold Ramis],2009-06-18T00:00:00Z,4.8,"[Adventure, Comedy]",https://m.media-amazon.com/images/M/MV5BMTcyMj...,"After being banished from their tribe, two hun...",Year One,2267,5820,"[Jack Black, Michael Cera, Olivia Wilde]",2009,tt1045778,add,"[being banished from their tribe, two hunter-g..."
1,movies,tt0075029,6.181381,[Clint Eastwood],1976-06-30T00:00:00Z,7.8,[Western],https://m.media-amazon.com/images/M/MV5BMTI2NT...,A Missouri farmer joins a Confederate guerrill...,The Outlaw Josey Wales,2683,8100,"[Clint Eastwood, Sondra Locke, Chief Dan George]",1976,tt0075029,add,[A Missouri farmer joins a Confederate guerril...
2,movies,tt0204175,5.870997,[Robert Iscove],2000-06-16T00:00:00Z,5.0,"[Comedy, Romance, Drama]",https://m.media-amazon.com/images/M/MV5BMTgyNj...,A friendship is put to the ultimate test when ...,Boys and Girls,4520,5640,"[Freddie Prinze Jr., Claire Forlani, Brendon R...",2000,tt0204175,add,[A friendship is put to the ultimate test when...


#### フレーズ検索
語句が特定の順序で並んでいるドキュメントのみを検索したい場合、フレーズ検索の機能が有用です。
以下のクエリは "iron man" を match query で検索した例ですが、"Iron man" シリーズだけではなく "The Man with the Iron Fists" などもヒットしています。

In [21]:
index_name = "movies"

payload = {
  "query": {
    "match": {
      "title": {
        "query": "iron man",
        "operator": "and"
      }
    }
  },
  "size": 5
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])


,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0371746,10.579328,[Jon Favreau],2008-04-14T00:00:00Z,7.9,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTczNT...,When wealthy industrialist Tony Stark is force...,Iron Man,171,7560,"[Robert Downey Jr., Gwyneth Paltrow, Terrence ...",2008,tt0371746,add
1,movies,tt1228705,9.705842,[Jon Favreau],2010-04-26T00:00:00Z,7.0,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0MD...,Tony Stark has declared himself Iron Man and i...,Iron Man 2,276,7440,"[Robert Downey Jr., Mickey Rourke, Gwyneth Pal...",2010,tt1228705,add
2,movies,tt1300854,9.431475,[Shane Black],2013-04-18T00:00:00Z,7.4,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMjIzMz...,When Tony Stark's world is torn apart by a for...,Iron Man 3,22,7800,"[Robert Downey Jr., Guy Pearce, Gwyneth Paltrow]",2013,tt1300854,add
3,movies,tt1258972,6.462547,[RZA],2012-11-02T00:00:00Z,5.4,[Action],https://m.media-amazon.com/images/M/MV5BMTg5OD...,"On the hunt for a fabled treasure of gold, a b...",The Man with the Iron Fists,513,5700,"[Russell Crowe, Cung Le, Lucy Liu]",2012,tt1258972,add
4,movies,tt0120744,6.152519,[Randall Wallace],1998-03-13T00:00:00Z,6.3,"[Action, Adventure]",https://m.media-amazon.com/images/M/MV5BMTczMD...,The cruel King Louis XIV of France has a secre...,The Man in the Iron Mask,1276,7920,"[Leonardo DiCaprio, Jeremy Irons, John Malkovich]",1998,tt0120744,add


[match_phrase][match_phrase] クエリを使用することで、特定の並び順でトークンが配置されているドキュメントのみを検索することが可能です。

[match_phrase]: https://opensearch.org/docs/latest/query-dsl/full-text/match-phrase/

In [22]:
index_name = "movies"

payload = {
  "query": {
    "match_phrase": {
      "title": {
        "query": "iron man",
      }
    }
  },
  "size": 5
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0371746,10.579328,[Jon Favreau],2008-04-14T00:00:00Z,7.9,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTczNT...,When wealthy industrialist Tony Stark is force...,Iron Man,171,7560,"[Robert Downey Jr., Gwyneth Paltrow, Terrence ...",2008,tt0371746,add
1,movies,tt1228705,9.705841,[Jon Favreau],2010-04-26T00:00:00Z,7.0,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0MD...,Tony Stark has declared himself Iron Man and i...,Iron Man 2,276,7440,"[Robert Downey Jr., Mickey Rourke, Gwyneth Pal...",2010,tt1228705,add
2,movies,tt1300854,9.431475,[Shane Black],2013-04-18T00:00:00Z,7.4,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMjIzMz...,When Tony Stark's world is torn apart by a for...,Iron Man 3,22,7800,"[Robert Downey Jr., Guy Pearce, Gwyneth Paltrow]",2013,tt1300854,add


slop パラメーターを変更することで、ワードの順序が入れ替わっていてもフレーズ検索にマッチさせることが可能です。デフォルトは 0 であるため、厳密なマッチングが要求されます。

slop を 2 に変更すると、2 つのキーワードが入れ替えの対象になります。

In [23]:
index_name = "movies"

payload = {
  "query": {
    "match_phrase": {
      "title": {
        "query": "man iron 2",
        "slop": 2
      }
    }
  },
  "size": 5
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1228705,6.408018,[Jon Favreau],2010-04-26T00:00:00Z,7,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0MD...,Tony Stark has declared himself Iron Man and i...,Iron Man 2,276,7440,"[Robert Downey Jr., Mickey Rourke, Gwyneth Pal...",2010,tt1228705,add


3 つのキーワードを入れ替えたい場合は、slop を 3 にセットします。

In [24]:
index_name = "movies"

payload = {
  "query": {
    "match_phrase": {
      "title": {
        "query": "man 2 iron",
        "slop": 3
      }
    }
  },
  "size": 5
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1228705,5.062435,[Jon Favreau],2010-04-26T00:00:00Z,7,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0MD...,Tony Stark has declared himself Iron Man and i...,Iron Man 2,276,7440,"[Robert Downey Jr., Mickey Rourke, Gwyneth Pal...",2010,tt1228705,add


### 完全一致検索 (Term query)
検索キーワードに完全に一致するドキュメントを取得する場合は、Term query を使用します。主に keyword タイプのフィールドに対する検索で使用します。
以下のクエリは、genres フィールドに "Comedy" という値を持つドキュメントを取得しています。検索結果の件数は size と呼ばれるパラメーターで制御可能です。

In [25]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "term": {
      "genres": {
        "value": "Comedy"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,1.498618,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add
1,movies,tt1690953,1.498618,"[Pierre Coffin, Chris Renaud]",2013-06-16T00:00:00Z,7.7,"[Animation, Adventure, Comedy, Crime, Family, ...",https://m.media-amazon.com/images/M/MV5BOTg4NT...,Gru is recruited by the Anti-Villain League to...,Despicable Me 2,55,5880,"[Steve Carell, Kristen Wiig, Benjamin Bratt]",2013,tt1690953,add
2,movies,tt0481499,1.498618,"[Kirk De Micco, Chris Sanders]",2013-02-15T00:00:00Z,7.3,"[Animation, Adventure, Comedy, Family, Fantasy]",https://m.media-amazon.com/images/M/MV5BMTcyOT...,"After their cave is destroyed, a caveman famil...",The Croods,91,5880,"[Nicolas Cage, Ryan Reynolds, Emma Stone]",2013,tt0481499,add


#### normalizer による正規化
完全一致検索の中でも、case-sensitive (大文字小文字の区別) の要否が分かれる場合があります。今回のようにジャンル名での検索であれば、Comedy ではなく comedy でも同様の結果を得られた方が好ましい場合があります。こうした要件に対応するために、OpenSearch では [Normalizer][normalizer] と呼ばれる機能を提供しています。Normalizer はデータ格納時、およびクエリ実行時に文字列の正規化を行う機能です。

本ワークショップでは、genres フィールドに lowercase_normalizer と呼ばれる Normalizer をセットしています。この Normalizer は lowercase filter により入力された文字を自動的に小文字に変換するよう設定されています。

Normalizer はインデックスの設定(settings/analysis)内で定義します。定義した Normalizer は、フィールドの normalizer オプションで指定することができます。以下はインデックス設定の抜粋です。

```json
{
  "settings": {
    "analysis": {
      "normalizer": {
        "lowercase_normalizer": {
          "type": "custom",
          "filter": ["lowercase"]
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "genres":  {
        "type": "keyword",
        "normalizer": "lowercase_normalizer"
      }
    }
  }
}
```

Normalizer により入力された文字が小文字に統一されるため、以下のように "cOmEdY" という文字列で一致検索を行っても、"Comedy" で検索した時と同様の結果が得られます。

[normalizer]: https://opensearch.org/docs/latest/analyzers/normalizers/

In [26]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "term": {
      "genres": {
        "value": "cOmEdY"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

print(json.dumps(response, indent=2))

{
  "took": 19,
  "timed_out": false,
  "_shards": {
    "total": 0,
    "successful": 0,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1776,
      "relation": "eq"
    },
    "max_score": 1.4986184,
    "hits": [
      {
        "_index": "movies",
        "_id": "tt1981677",
        "_score": 1.4986184,
        "_source": {
          "directors": [
            "Jason Moore"
          ],
          "release_date": "2012-09-28T00:00:00Z",
          "rating": 7.1,
          "genres": [
            "Comedy",
            "Music",
            "Romance"
          ],
          "image_url": "https://m.media-amazon.com/images/M/MV5BMTcyMTMzNzE5N15BMl5BanBnXkFtZTcwNzg5NjM5Nw@@._V1_SX400_.jpg",
          "plot": "Beca, a freshman at Barden University, is cajoled into joining The Bellas, her school's all-girls singing group. Injecting some much needed energy into their repertoire, The Bellas take on their male rivals in a campus competition.",
          "title": 

#### Terms
指定した複数の検索条件のいずれかに完全一致するドキュメントを取得したい場合は、term ではなく [terms][terms] クエリを使用します

以下のクエリでは comedy もしくは drama のジャンルの映画を検索しています

[terms]: https://opensearch.org/docs/latest/query-dsl/term/terms/

In [27]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "terms": {
      "genres": ["comedy", "drama"]
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt2024544,1.0,[Steve McQueen],2013-08-30T00:00:00Z,7.7,"[Biography, Drama, History]",https://m.media-amazon.com/images/M/MV5BMjExMT...,"In the antebellum United States, Solomon North...",12 Years a Slave,44,7980,"[Chiwetel Ejiofor, Michael K. Williams, Michae...",2013,tt2024544,add
1,movies,tt1981677,1.0,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add
2,movies,tt1758595,1.0,[Oliver Hirschbiegel],2013-09-05T00:00:00Z,4.6,"[Biography, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjA0ND...,"During the last two years of her life, Princes...",Diana,54,6780,"[Naomi Watts, Naveen Andrews, Douglas Hodge]",2013,tt1758595,add


#### Terms set
Terms は配列内のいずれかの文字列と完全一致していればマッチした文書とみなして検索結果を返していました。一方で、配列内のすべて、もしくは一部の文字列と完全一致している場合にマッチしたとみなしたいケースもあります。
こうしたケースでは Terms set クエリが有用です。以下は comedy, drama, sci-fi, family の要素をすべて含むジャンルの映画を検索するものです

In [28]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "terms_set": {
      "genres": {
        "terms": ["comedy", "drama", "sci-fi", "family"],
        "minimum_should_match_script": {
          "source": "params.num_terms"
        }
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0100758,8.298736,[Steve Barron],1990-03-30T00:00:00Z,6.4,"[Action, Adventure, Comedy, Crime, Drama, Fami...",https://m.media-amazon.com/images/M/MV5BMTk3OT...,A quartet of mutated humanoid turtles clash wi...,Teenage Mutant Ninja Turtles,440,5580,"[Judith Hoag, Elias Koteas, Josh Pais]",1990,tt0100758,add


#### Terms lookup
[Terms lookup][terms-lookup] を利用することで、他インデックスのデータを使用した Terms query が実行可能です。 

以下のサンプルコードでは、ユーザーごとのお気に入り情報を格納した movie-users インデックスを作成・データを格納し、ユーザーに対応したお気に入りジャンルの映画を検索しています

[terms-lookup]: https://opensearch.org/docs/latest/query-dsl/term/terms/#terms-lookup

In [29]:
lookup_index_name = "movie-users"

payload = {
  "settings": {
    "index": {
      "number_of_shards": 1,
      "number_of_replicas": 0
    }
  },
  "mappings": {
    "properties": {
      "userid":  { "type": "keyword" },
      "favorite-genres":  { "type": "keyword" }
    }
  }
}

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=lookup_index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスの作成を行う
print("# create index")
response = opensearch_client.indices.create(index=lookup_index_name, body=payload)
print(json.dumps(response, indent=2))

# delete index
{
  "acknowledged": true
}
# create index
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "movie-users"
}


In [30]:
lookup_index_name = "movie-users"

payload = {
  "userid": "00000001",
  "favorite-genres": ["comedy", "drama"]
}

response = opensearch_client.index(
    index = lookup_index_name,
    body = payload,
    id = "00000001",
    # AOSS では、refresh は使用できない
    # refresh = True
)

print(json.dumps(response, indent=2))

{
  "_index": "movie-users",
  "_id": "00000001",
  "_version": 1,
  "result": "created",
  "_shards": {
    "total": 0,
    "successful": 0,
    "failed": 0
  },
  "_seq_no": 0,
  "_primary_term": 0
}


In [31]:
index_name = "movies"
lookup_index_name = "movie-users"

payload = {
  "size": 3,
  "query": {
    "terms": {
      "genres": {
        "index": lookup_index_name,
        "id": "00000001",
        "path": "favorite-genres"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

""


<div class="alert alert-block alert-info"> 
<b>Tips: 参照させる要素数が多い場合の対応</b>
    
Terms lookup で与える要素数が 10,000 を超える場合は、パフォーマンス向上のために [Bitmap Filtering][bitmap-filtering] の利用を検討してください。

[bitmap-filtering]: https://opensearch.org/docs/latest/query-dsl/term/terms/#bitmap-filtering
</div>



### 部分一致検索・あいまい検索
Keyword フィールドに対して部分的に一致する検索条件を記載したい場合や、多少の表記ゆれや誤字脱字をフォローした検索を行いたい場合の検索手法について解説します。

全ドキュメントに対して横断的に検索が実行されるため、通常の term クエリと比較してパフォーマンスは大きく劣ります。

<div class="alert alert-block alert-warning"> 
<b>あいまい検索利用時の注意事項</b>

これらの検索は、インデックス内の全ドキュメントが検索対象となる可能性があり、パフォーマンスへの影響があります。

対象フィールドに対する検索クエリの大部分を部分一致検索やあいまい検索が占める場合は、text 型フィールドを使用した全文検索に切り替えるか、対象のフィールドを [wildcard][wildcard] 型に変更することを検討してください。

</div>


[wildcard]: https://opensearch.org/docs/latest/field-types/supported-field-types/wildcard/

#### Prefix query
[Prefix query][prefix] は、前方一致に基づく検索を行うクエリです。特定の文字列から始まるテキストを検索する際に役立ちます。

以下のサンプルクエリでは、com から始まる genres の映画を検索します。

[prefix]: https://opensearch.org/docs/latest/query-dsl/term/prefix/

In [32]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "prefix": {
      "genres": "com"
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,1.0,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add
1,movies,tt1690953,1.0,"[Pierre Coffin, Chris Renaud]",2013-06-16T00:00:00Z,7.7,"[Animation, Adventure, Comedy, Crime, Family, ...",https://m.media-amazon.com/images/M/MV5BOTg4NT...,Gru is recruited by the Anti-Villain League to...,Despicable Me 2,55,5880,"[Steve Carell, Kristen Wiig, Benjamin Bratt]",2013,tt1690953,add
2,movies,tt0481499,1.0,"[Kirk De Micco, Chris Sanders]",2013-02-15T00:00:00Z,7.3,"[Animation, Adventure, Comedy, Family, Fantasy]",https://m.media-amazon.com/images/M/MV5BMTcyOT...,"After their cave is destroyed, a caveman famil...",The Croods,91,5880,"[Nicolas Cage, Ryan Reynolds, Emma Stone]",2013,tt0481499,add


#### Regexp query
Regexp query は、正規表現に基づく検索を行うクエリです。特定のパターンに一致する文字列を抽出する際に使用できます。

以下のサンプルクエリでは、アルファベット 5 文字の genres の映画を検索します。

[regexp]: https://opensearch.org/docs/latest/query-dsl/term/regexp/

In [33]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "regexp": {
      "genres": "[a-z]{5}"
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1670345,1.0,[Louis Leterrier],2013-05-21T00:00:00Z,7.3,"[Crime, Mystery, Thriller]",https://m.media-amazon.com/images/M/MV5BMTY0ND...,An FBI agent and an Interpol detective track a...,Now You See Me,11,6900,"[Jesse Eisenberg, Common, Mark Ruffalo]",2013,tt1670345,add


#### Wildcard query
[Wildcard query][wildcard] は、ワイルドカードを使用した部分一致検索を提供します。以下のクエリでは、 c から始まって y で終わる genres の映画を検索します。

[wildcard]: https://opensearch.org/docs/latest/query-dsl/term/wildcard/

In [34]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "wildcard": {
      "genres": "c*y"
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,1.0,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


#### Fuzzy query
[Fuzzy query][fuzzy] は、以下のような表記ゆれをクエリ側で吸収する機能です。

- 誤字: **c**at to **b**at
- 不要文字の混入: cat to cat**s**
- 脱字: **c**at to at
- タイプミス(順序の入れ替わり): **ca**t to **ac**t

[fuzzy]: https://opensearch.org/docs/latest/query-dsl/term/fuzzy/

以下のサンプルクエリでは、comedy から二文字かけた cmdy でも検索にヒットするように fuziness を 2 にセットして Fuzzy query を実行しています。

fuziness はデフォルトで `AUTO` という値がセットされており、OpenSearch 側で語句の長さから自動的に補完文字数をセットする仕様になっています。fuziness に数値を明示的にセットすることで、何文字まで補完されるかをコントロールすることが可能です。

In [35]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "cmdy",
        "fuzziness": 2
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,0.749309,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


以下のサンプルクエリでは、comedy のタイプミスである ocmedy で検索しています。

タイプミスにより隣り合った文字が入れ替わったケースに対応するかどうかは、**transpositions** オプションで制御します。同オプションのデフォルト値は **true** となっているため、Fuzzy query のデフォルトの動作としては、タイプミスをフォローする形になっているといえます。

In [36]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "ocmedy",
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,1.248849,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


文字の入れ替わりを何か所まで許容するかは、**fuzziness** パラメーターで制御可能です。

ocemdy で comedy が含まれるドキュメントをヒットさせたい場合、**fuzziness** が `1` ではヒットしません。`2` 以上ではヒットします

In [37]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "ocemdy",
        "fuzziness": 1
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

""


In [38]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "ocemdy",
        "fuzziness": 2
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,0.999079,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


タイプミスを Fuzzy query でフォローしない場合、**transpositions** に `false` をセットします。

In [123]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "ocemdy",
        "fuzziness": 2,
        "transpositions": False
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

""


transpositions による補正は隣り合った文字同士でのみ機能します。mocedy のように飛び石で入れ替わってしまったケースは対応できません。

In [39]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "mocedy",
        "fuzziness": 1,
        "transpositions": True
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

""


このようなケースでも、Fuzzy query は誤字の補正対応で対応可能です。 fuzziness の値に対応した文字数までであれば、入れ替わりや誤字に対して対処可能です。

In [40]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "mocedy",
        "fuzziness": 2,
        "transpositions": False
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,0.999079,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


以下のケースでは、入れ替わりに加えて m ではなく n のタイプミスも加わっていますが、**n**o**c**edy のうち n と c の 2 文字が fuzziness = 2 の設定により補完され、**c**o**m**edy とマッチすると判定されています。

In [41]:
index_name = "movies"

payload = {
  "size": 1,
  "query": {
    "fuzzy": {
      "genres": {
        "value": "nocedy",
        "fuzziness": 2,
        "transpositions": False
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1981677,0.999079,[Jason Moore],2012-09-28T00:00:00Z,7.1,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add


### 検索結果のソート
本セクションでは、検索結果の並べ替え手法について解説します。

#### スコアによるソート
OpenSearch では、[キーワード検索][keyword-search]における文書の関連度計算に [Okapi BM25][okapi-bm25] を使用しています。BM25 は、クエリ内に出現する単語の語彙検索を実行するキーワードベースのアルゴリズムです。

文書の関連性を判断する際、BM25 は用語頻度(TF - Term Frequency) および 逆文書頻度 (IDF - Inverse Document Frequency) を考慮します。

TF(用語頻度)は、特定文書における、特定の単語の出現頻度を示します。検索対象の用語がより頻繁に出現する文書を、関連性が高い文書として扱います。
一方、IDF(逆文書頻度)は、コーパス内のすべての文書に共通して頻出する単語の重みを低くするものです。the や a のような冠詞が該当します。

<img src="./img/bm25.png" width=1024>

[keyword-search]: https://opensearch.org/docs/latest/search-plugins/keyword-search/
[okapi-bm25]:https://en.wikipedia.org/wiki/Okapi_BM25

計算後の score は "_score" フィールドに格納されています。Search API 実行時に [explain][explain] オプションを付与することで、詳細なスコア計算の過程を確認することが可能です。

[explain]: https://opensearch.org/docs/latest/api-reference/explain/

In [42]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "match": {
      "plot": "superhero"
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "explain": True
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_shard,_node,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,...,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot,_explanation.value,_explanation.description,_explanation.details
0,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.165.128,movies,tt0383060,7.451531,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,...,1395,5580.0,"[Tim Allen, Courteney Cox, Chevy Chase]",2006.0,tt0383060,add,[Former <em>superhero</em> Jack is called back...,7.451531,weight(plot:superhero in 179) [PerFieldSimilar...,"[{'value': 7.451531, 'description': 'score(fre..."
1,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.123.128,movies,tt0132347,7.161061,[Kinka Usher],1999-07-22T00:00:00Z,5.9,"[Action, Comedy, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM2Nj...,...,2279,7260.0,"[Ben Stiller, Janeane Garofalo, William H. Macy]",1999.0,tt0132347,add,[A group of inept amateur <em>superheroes</em>...,7.161061,weight(plot:superhero in 163) [PerFieldSimilar...,"[{'value': 7.1610613, 'description': 'score(fr..."
2,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.123.128,movies,tt0451279,6.797963,NaN,NaN,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",NaN,...,2806,NaN,NaN,NaN,tt0451279,add,[An Amazon princess comes to the world of Man ...,6.797963,weight(plot:superhero in 368) [PerFieldSimilar...,"[{'value': 6.797963, 'description': 'score(fre..."


#### 重みづけによるスコアの調整
Multi match query など、複数フィールドに対して横断的に検索を行う場合、フィールドごとに重みづけを設定することが可能です。以下の例では、title と plot に対する横断検索を行う際、title フィールドの重みを 3 倍にしています。

In [43]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "multi_match": {
      "query": "wind",
      "fields": ["title^3", "plot"]
    }
  },
  "highlight": {
    "fields": {
      "title": {},
      "plot": {}
    }
  },
  "explain": True
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_shard,_node,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,...,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.title,_explanation.value,_explanation.description,_explanation.details,highlight.plot
0,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.165.128,movies,tt0031381,15.168678,"[Victor Fleming, George Cukor, Sam Wood]",1939-12-15T00:00:00Z,8.2,"[Drama, Romance, War]",https://m.media-amazon.com/images/M/MV5BNDUwMj...,...,14280,"[Clark Gable, Vivien Leigh, Thomas Mitchell]",1939,tt0031381,add,[Gone with the <em>Wind</em>],15.168678,max of:,"[{'value': 15.168678, 'description': 'weight(t...",NaN
1,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.123.128,movies,tt0460989,11.959131,[Ken Loach],2006-05-18T00:00:00Z,7.4,"[Drama, History, War]",https://m.media-amazon.com/images/M/MV5BMTc1Mj...,...,7620,"[Cillian Murphy, Padraic Delaney, Liam Cunning...",2006,tt0460989,add,[The <em>Wind</em> That Shakes the Barley],11.959131,max of:,"[{'value': 11.959131, 'description': 'weight(t...",NaN
2,[720701806597::bjb702xlcqxfnz2c2hbf::SEARCH::m...,10.0.123.128,movies,tt1045778,6.200220,[Harold Ramis],2009-06-18T00:00:00Z,4.8,"[Adventure, Comedy]",https://m.media-amazon.com/images/M/MV5BMTcyMj...,...,5820,"[Jack Black, Michael Cera, Olivia Wilde]",2009,tt1045778,add,NaN,6.200220,max of:,"[{'value': 6.20022, 'description': 'weight(plo...","[being banished from their tribe, two hunter-g..."


#### スクリプトによるスコアの調整
[Script score][script-score] クエリを使用することで、スコア計算を script ベースで行うことも可能です。以下の例では、関連度に基づいて算出された score を rank の値で割った値を最終的な score としています。

[script-score]: https://opensearch.org/docs/latest/query-dsl/specialized/script-score/

In [44]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "script_score": {
      "query": {
        "match": { 
            "plot": "superhero" 
        }
      },
      "script": {
        "source": "_score / doc['rank'].value"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0317705,0.053698,[Brad Bird],2004-10-27T00:00:00Z,8.0,"[Animation, Action, Adventure, Family]",https://m.media-amazon.com/images/M/MV5BMTY5OT...,"A family of undercover superheroes, while tryi...",The Incredibles,112,6900,"[Craig T. Nelson, Samuel L. Jackson, Holly Hun...",2004,tt0317705,add
1,movies,tt0458339,0.019516,[Joe Johnston],2011-07-19T00:00:00Z,6.8,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTYzOT...,"After being deemed unfit for military service,...",Captain America: The First Avenger,253,7440,"[Chris Evans, Hugo Weaving, Samuel L. Jackson]",2011,tt0458339,add
2,movies,tt0409459,0.016800,[Zack Snyder],2009-02-23T00:00:00Z,7.6,"[Action, Drama, Mystery, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTc0Nj...,In an alternate 1985 where former superheroes ...,Watchmen,304,9720,"[Jackie Earle Haley, Patrick Wilson, Carla Gug...",2009,tt0409459,add


Script score では sigmoid といったいくつかの関数も提供しています。

In [45]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "script_score": {
      "query": {
        "match": { 
            "plot": "superhero" 
        }
      },
      "script": {
        "source": "sigmoid(_score, 2, 1)"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0383060,0.788394,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,Former superhero Jack is called back to work t...,Zoom,1395,5580.0,"[Tim Allen, Courteney Cox, Chevy Chase]",2006.0,tt0383060,add
1,movies,tt0132347,0.781685,[Kinka Usher],1999-07-22T00:00:00Z,5.9,"[Action, Comedy, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM2Nj...,A group of inept amateur superheroes must try ...,Mystery Men,2279,7260.0,"[Ben Stiller, Janeane Garofalo, William H. Macy]",1999.0,tt0132347,add
2,movies,tt0451279,0.772675,NaN,NaN,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",NaN,An Amazon princess comes to the world of Man t...,Wonder Woman,2806,NaN,NaN,NaN,tt0451279,add


#### キーワードによる Boosting
スコアの補正は検索キーワードに対しても適用することが可能です。[Boosting][boosting] クエリを使用することで、negative に指定したキーワードにマッチするドキュメントのスコアを下げて検索順位を変動させることができます。以下の例では、private という文字列を plot フィールドに含むドキュメントのスコアを 0.9 倍にセットしています。

[boosting]: https://opensearch.org/docs/latest/query-dsl/compound/boosting/

In [131]:
index_name = "movies"

payload = {
  "size": 3,
  "query": {
    "boosting": {
      "positive": {
        "match": {
          "plot": "superhero"
        }
      },
      "negative": {
        "match": {
          "plot": "private"
        }
      },
      "negative_boost": 0.9
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0132347,7.161061,[Kinka Usher],1999-07-22T00:00:00Z,5.9,"[Action, Comedy, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM2Nj...,A group of inept amateur superheroes must try ...,Mystery Men,2279,7260.0,"[Ben Stiller, Janeane Garofalo, William H. Macy]",1999.0,tt0132347,add
1,movies,tt0451279,6.797963,NaN,NaN,NaN,"[Action, Adventure, Fantasy, Sci-Fi]",NaN,An Amazon princess comes to the world of Man t...,Wonder Woman,2806,NaN,NaN,NaN,tt0451279,add
2,movies,tt0383060,6.706378,[Peter Hewitt],2006-08-11T00:00:00Z,3.9,"[Action, Adventure, Family, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTM0Nj...,Former superhero Jack is called back to work t...,Zoom,1395,5580.0,"[Tim Allen, Courteney Cox, Chevy Chase]",2006.0,tt0383060,add


#### 特定フィールドによるソート
スコアではなく特定のフィールドに基づいてソートを行いたい場合は、[sort][sort] オプションを検索 API に追加します。
以下のサンプルでは、公開年度が新しい映画の上位 3 位を取得しています。query オプションで指定している match_all は、全てのドキュメントが取得対象であることを示します。また上位 5 位だけを取得するために、size オプションに 5 を設定しています。

[sort]: https://opensearch.org/docs/latest/search-plugins/searching-data/sort/

In [46]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "match_all": {}
  },
  "sort": [
    {
      "year": {
        "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.genres,_source.plot,_source.title,_source.rank,_source.year,_source.id,_source.type,_source.release_date,_source.actors
0,movies,tt1502407,None,[2018],[Patrick Lussier],[Horror],The plot is unknown at this time.,Halloween III,4896,2018,tt1502407,add,NaN,NaN
1,movies,tt0974015,None,[2017],NaN,"[Action, Adventure, Fantasy, Sci-Fi]",The world's greatest heroes are assembled to f...,Justice League,1295,2017,tt0974015,add,2017-01-01T00:00:00Z,NaN
2,movies,tt0439572,None,[2016],[Greg Berlanti],"[Action, Adventure, Drama, Fantasy, Sci-Fi]",NaN,The Flash,2281,2016,tt0439572,add,2016-01-01T00:00:00Z,NaN
3,movies,tt1790809,None,[2016],"[Joachim Rønning, Espen Sandberg]","[Action, Adventure, Comedy, Fantasy]",The fifth installment of the blockbuster franc...,Pirates of the Caribbean: Dead Men Tell No Tales,384,2016,tt1790809,add,NaN,[Johnny Depp]
4,movies,tt1630029,None,[2016],[James Cameron],"[Action, Adventure, Fantasy, Sci-Fi]",A sequel to the 2009 global phenomenon.,Avatar 2,909,2016,tt1630029,add,2016-12-01T00:00:00Z,"[Zoe Saldana, Sigourney Weaver, Sam Worthington]"


sort には複数条件を記載することができます。上記のクエリでは 2016 年の映画が下位 3 件を占めていました。2016 年に公開された映画を ID 順に並び替えたいと思います。<br>
以下のように year によるソート条件の下に id によるソート条件を追加することで、year によるソートの後に id によるソートを実行することが可能です。複数条件で sort を行う場合は、先に実行したいソート条件を上に記載します。

In [47]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "match_all": {}
  },
  "sort": [
    {
      "year": {
        "order": "desc"
      }
    },
    {
      "id": {
        "order": "asc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.genres,_source.plot,_source.title,_source.rank,_source.year,_source.id,_source.type,_source.release_date,_source.actors
0,movies,tt1502407,None,"[2018, tt1502407]",[Patrick Lussier],[Horror],The plot is unknown at this time.,Halloween III,4896,2018,tt1502407,add,NaN,NaN
1,movies,tt0974015,None,"[2017, tt0974015]",NaN,"[Action, Adventure, Fantasy, Sci-Fi]",The world's greatest heroes are assembled to f...,Justice League,1295,2017,tt0974015,add,2017-01-01T00:00:00Z,NaN
2,movies,tt0439572,None,"[2016, tt0439572]",[Greg Berlanti],"[Action, Adventure, Drama, Fantasy, Sci-Fi]",NaN,The Flash,2281,2016,tt0439572,add,2016-01-01T00:00:00Z,NaN
3,movies,tt1630029,None,"[2016, tt1630029]",[James Cameron],"[Action, Adventure, Fantasy, Sci-Fi]",A sequel to the 2009 global phenomenon.,Avatar 2,909,2016,tt1630029,add,2016-12-01T00:00:00Z,"[Zoe Saldana, Sigourney Weaver, Sam Worthington]"
4,movies,tt1790809,None,"[2016, tt1790809]","[Joachim Rønning, Espen Sandberg]","[Action, Adventure, Comedy, Fantasy]",The fifth installment of the blockbuster franc...,Pirates of the Caribbean: Dead Men Tell No Tales,384,2016,tt1790809,add,NaN,[Johnny Depp]


### 範囲検索
数値や日付を元に連続した範囲で検索を行いたい場合は [Range query][range] を使用します。 以下の例では 1920 年から 1923 年に公開された映画を検索しています。

[range]: https://opensearch.org/docs/latest/query-dsl/term/range/

In [48]:
index_name = "movies"

payload = {
  "query": {
    "range": {
      "year": {
        "gte": 1920,
        "lte": 1923
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0012349,1.0,[Charles Chaplin],1921-01-21T00:00:00Z,8.3,"[Comedy, Drama, Family]",https://m.media-amazon.com/images/M/MV5BMTg2Nj...,"The Tramp cares for an abandoned child, but ev...",The Kid,4925,4080,"[Charles Chaplin, Edna Purviance, Jackie Coogan]",1921,tt0012349,add
1,movies,tt0013442,1.0,[F.W. Murnau],1922-02-17T00:00:00Z,8.0,[Horror],https://m.media-amazon.com/images/M/MV5BMTYyNj...,Vampire Count Orlok expresses interest in a ne...,"Nosferatu, eine Symphonie des Grauens",2993,5640,"[Max Schreck, Greta Schröder, Ruth Landshoff]",1922,tt0013442,add
2,movies,tt0010323,1.0,[Robert Wiene],1920-02-26T00:00:00Z,8.0,[Horror],https://m.media-amazon.com/images/M/MV5BMTQwMz...,"Dr. Caligari's somnambulist, Cesare, and his d...",Das Cabinet des Dr. Caligari,4950,4680,"[Werner Krauss, Conrad Veidt, Friedrich Feher]",1920,tt0010323,add


### 複合検索
[Boolean query][bool] を利用することで、複数の条件を組み合わせた検索を行うことが可能です。以下の要素を使用します。

- must: 条件に全て合致するドキュメントを返す
- should: 1 つ以上の条件に合致するドキュメントを返す
- filter: 条件に全て合致するドキュメントを返す。must と違って score 計算に関与しない。
- must_not: 条件に合致するドキュメントを検索結果から除外する

[bool]: https://opensearch.org/docs/latest/query-dsl/compound/bool/

#### Boolean query による複合検索
実際に複合条件による検索を行っていきます。まず、クエリに含まれる条件を列挙します

- must: plot に school が含まれること
- must: genres に Music が必ず含まれること。
- should: genres に Romance または Comedy のいずれか 1 つ (minimum_should_match にて指定)が含まれていること
- must_not: genres に Thriller または Horror を含まないこと
- filter: year が 2000 から 2009 の範囲であること
- sort: 検索結果を rating フィールドで降順に並べ替えること
- size: 検索結果のうち上位 5 件を出力すること



In [49]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0332379,None,[7.0],[Richard Linklater],2003-09-09T00:00:00Z,7.0,"[Comedy, Music]",https://m.media-amazon.com/images/M/MV5BMjEwOT...,A wannabe rock star in need of cash poses as a...,The School of Rock,727,6480,"[Jack Black, Mike White, Joan Cusack]",2003,tt0332379,add,[A wannabe rock star in need of cash poses as ...
1,movies,tt0981227,None,[6.7],[Peter Sollett],2008-09-06T00:00:00Z,6.7,"[Comedy, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BOTY2Mj...,"High school student Nick O'Leary, member of th...",Nick and Norah's Infinite Playlist,1701,5400,"[Michael Cera, Kat Dennings, Aaron Yoo]",2008,tt0981227,add,"[High <em>school</em> student Nick O'Leary, me..."
2,movies,tt0462590,None,[6.2],[Anne Fletcher],2006-08-07T00:00:00Z,6.2,"[Crime, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTIxMD...,Tyler Gage receives the opportunity of a lifet...,Step Up,947,6240,"[Channing Tatum, Jenna Dewan-Tatum, Damaine Ra...",2006,tt0462590,add,[Tyler Gage receives the opportunity of a life...
3,movies,tt1023481,None,[5.800000000000001],[Jon M. Chu],2008-02-14T00:00:00Z,5.8,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTc0Nz...,Romantic sparks occur between two dance studen...,Step Up 2: The Streets,1427,5880,"[Robert Hoffman, Briana Evigan, Cassie Ventura]",2008,tt1023481,add,[Romantic sparks occur between two dance stude...
4,movies,tt0361696,None,[5.5],[Sean McNamara],2004-10-03T00:00:00Z,5.5,"[Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTM1MT...,A girl from a small town heads to the big city...,Raise Your Voice,3425,6180,"[Hilary Duff, John Corbett, Rebecca De Mornay]",2004,tt0361696,add,[from a small town heads to the big city of Lo...


#### Zero terms query の取り扱い

検索においては、match クエリに与えられるトークンが実質的に 0 件という状況が発生します。例えば以下のようなケースが考えられます。

- ユーザーが入力するクエリ文字列がストップワードのみで構成されているため、アナライザーによって全てのトークンが削除されてしまう ("To be or not to be" など)
- そもそもクエリが入力されておらず、フィルタ条件のみが指定されている

この場合、OpenSearch のデフォルトの挙動は検索を行わない、つまり 0 件ヒットとなります。

しかしながら、キーワードが入力されない場合はフィルタ条件として入力したジャンルや公開年に基づいて絞り込まれた映画の、rating が高いものを返却するのが自然な挙動と考えられます。<br>
この挙動は、[zero_terms_query][zero_terms_query] のオプションに all と指定することで実現可能です。以下 2 つのクエリを実行することで、zero_terms_query の設定による結果の差異を確認することが可能です。1 つ目のクエリは zero_terms_query オプションにデフォルトの none を、2 つ目のクエリは all をセットしています。

</div>

[zero_terms_query]: https://opensearch.org/docs/latest/query-dsl/full-text/match/#empty-query

In [50]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "",
              "zero_terms_query": "none"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

""


In [51]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "",
              "zero_terms_query": "all"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0907657,None,[7.9],[John Carney],2006-07-15T00:00:00Z,7.9,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTEwNj...,A modern-day musical about a busker and an imm...,Once,1863,5100,"[Glen Hansard, Markéta Irglová, Hugh Walsh]",2006,tt0907657,add
1,movies,tt0358273,None,[7.800000000000001],[James Mangold],2005-09-04T00:00:00Z,7.8,"[Biography, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMjIyOT...,A chronicle of country music legend Johnny Cas...,Walk the Line,794,8160,"[Joaquin Phoenix, Reese Witherspoon, Ginnifer ...",2005,tt0358273,add
2,movies,tt0857191,None,[7.7],[Thomas McCarthy],2007-09-07T00:00:00Z,7.7,"[Crime, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTIzNT...,A college professor travels to New York City t...,The Visitor,4743,6240,"[Richard Jenkins, Haaz Sleiman, Danai Gurira]",2007,tt0857191,add
3,movies,tt0249462,None,[7.6000000000000005],[Stephen Daldry],2000-05-19T00:00:00Z,7.6,"[Comedy, Drama, Music]",https://m.media-amazon.com/images/M/MV5BMjA3Mz...,A talented young boy becomes torn between his ...,Billy Elliot,2268,6600,"[Jamie Bell, Julie Walters, Jean Heywood]",2000,tt0249462,add
4,movies,tt0146882,None,[7.5],[Stephen Frears],2000-03-17T00:00:00Z,7.5,"[Comedy, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTgxMT...,"Rob, a record store owner and compulsive list ...",High Fidelity,2134,6780,"[John Cusack, Iben Hjejle, Todd Louiso]",2000,tt0146882,add


### 集計 (Aggregation)
ユーザーが検索を行う際、キーワードを入力して検索する方法とは別に、ジャンルなどの分類を元に絞り込んでいく方法があります。このような検索方法をファセットナビゲーションやファセット検索と呼びます。

ファセットナビゲーションにより、ユーザーはどのような分類が存在するのか、各分類ごとに検索対象のドキュメント件数がどの程度存在するかを事前に確認することが可能です。またおおまかな検索結果に対してファセットナビゲーションを併用することで検索結果のフィルタリングを行うことも可能です。

[AWS のドキュメント検索](https://docs.aws.amazon.com/search/doc-search.html?searchPath=documentation&searchQuery=OpenSearch) でもファセットナビゲーションが採用されています。上部のプルダウンメニューには検索結果から得られたサービス一覧などがあり、このメニューからサービスごとのドキュメントに絞った検索を行うことができます。

<img src="./img/facet.png" width=1024>

OpenSearch では、[Aggregations] クエリで集計を実行します。Aggregations は、ファセット検索で有用なキーワードの集計から、平均値(avg) や最大値(max) など統計値を出力するなど様々な集計方法を提供しています。Aggregations クエリを使用する場合、Search API で aggs オプションを使用します

以下のサンプルでは、plot に school を含みかつ genres に Music を持つ映画について、ジャンルごとのドキュメント件数を集計しています。

[aggregations]: https://opensearch.org/docs/latest/aggregations/


In [52]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "school"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ],
  "aggs": {
    "genres": {
      "terms": {
        "field": "genres"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["aggregations"]["genres"]["buckets"])

,key,doc_count
0,music,14
1,drama,8
2,romance,8
3,comedy,6
4,family,4
5,adventure,1
6,animation,1
7,crime,1
8,fantasy,1


上記の集計結果を見て、family もフィルタリング対象のジャンルに追加したとします。この場合、以下のようにクエリを書き換えて実行することで、ファセットによるドリルダウン相当の処理を実現できます。

In [53]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "school"
            }
          }
        },
        {
          "terms_set": {
            "genres": {
              "terms": ["music", "family"],
              "minimum_should_match_script": {
                "source": "params.num_terms"
              }
            }
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ],
  "aggs": {
    "genres": {
      "terms": {
        "field": "genres"
      }
    }
  }
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["aggregations"]["genres"]["buckets"])

,key,doc_count
0,family,4
1,music,4
2,comedy,3
3,romance,2
4,adventure,1
5,animation,1
6,fantasy,1


### ページング
アプリケーション要件によっては、数千件の検索結果を扱うケースがあります。多量のデータを一括で返却することはパフォーマンス上の悪影響があるため、Opensearch では多数の結果を少量ずつ取得する[ページング処理][paginate]を実装することを推奨しています。

ページングの実装方法は以下の 4 パターンが存在します。

- from + size: 10000 件以下の検索結果における簡易なページング
- scroll : 多数のドキュメントをバッチ処理
- search_after
- search_after + PIT

[paginate]: https://opensearch.org/docs/latest/search-plugins/searching-data/paginate/

#### from + size によるページング
[from + size][the-from-and-size-parameters] は最もシンプルな実装方法です。 from に指定した開始位置から size 件数分のデータを返却します。from に 100、size に 20 と指定した場合は、先頭から数えて 100 件目から 20 件のデータが返却されます。SQL の OFFSET と LIMIT の組み合わせと似ているところがあります。

以下のサンプルでは、8 件の検索結果を 2 つのクエリで分割取得しています。1 つめのクエリで from に 0(デフォルト)、size に 4 を指定し先頭 4 件の検索結果を、2 つめのクエリで from に 5、size に 4 を指定して後続の 4 件の検索結果を取得しています。

[the-from-and-size-parameters]: https://opensearch.org/docs/latest/search-plugins/searching-data/paginate/#the-from-and-size-parameters

In [54]:
index_name = "movies"

payload = {
  "from": 0,
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0332379,None,[7.0],[Richard Linklater],2003-09-09T00:00:00Z,7.0,"[Comedy, Music]",https://m.media-amazon.com/images/M/MV5BMjEwOT...,A wannabe rock star in need of cash poses as a...,The School of Rock,727,6480,"[Jack Black, Mike White, Joan Cusack]",2003,tt0332379,add,[A wannabe rock star in need of cash poses as ...
1,movies,tt0981227,None,[6.7],[Peter Sollett],2008-09-06T00:00:00Z,6.7,"[Comedy, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BOTY2Mj...,"High school student Nick O'Leary, member of th...",Nick and Norah's Infinite Playlist,1701,5400,"[Michael Cera, Kat Dennings, Aaron Yoo]",2008,tt0981227,add,"[High <em>school</em> student Nick O'Leary, me..."
2,movies,tt0462590,None,[6.2],[Anne Fletcher],2006-08-07T00:00:00Z,6.2,"[Crime, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTIxMD...,Tyler Gage receives the opportunity of a lifet...,Step Up,947,6240,"[Channing Tatum, Jenna Dewan-Tatum, Damaine Ra...",2006,tt0462590,add,[Tyler Gage receives the opportunity of a life...
3,movies,tt1023481,None,[5.800000000000001],[Jon M. Chu],2008-02-14T00:00:00Z,5.8,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTc0Nz...,Romantic sparks occur between two dance studen...,Step Up 2: The Streets,1427,5880,"[Robert Hoffman, Briana Evigan, Cassie Ventura]",2008,tt1023481,add,[Romantic sparks occur between two dance stude...


In [55]:
payload = {
  "from": 4,
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0361696,None,[5.5],[Sean McNamara],2004-10-03T00:00:00Z,5.5,"[Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTM1MT...,A girl from a small town heads to the big city...,Raise Your Voice,3425,6180,"[Hilary Duff, John Corbett, Rebecca De Mornay]",2004,tt0361696,add,[from a small town heads to the big city of Lo...
1,movies,tt0306841,None,[4.9],[Jim Fall],2003-04-26T00:00:00Z,4.9,"[Adventure, Comedy, Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTMzND...,Lizzie McGuire has graduated from middle schoo...,The Lizzie McGuire Movie,3219,5640,"[Hilary Duff, Adam Lamberg, Clayton Snyder]",2003,tt0306841,add,[Lizzie McGuire has graduated from middle <em>...
2,movies,tt1231580,None,[4.2],[Betty Thomas],2009-12-11T00:00:00Z,4.2,"[Animation, Comedy, Family, Fantasy, Music]",https://m.media-amazon.com/images/M/MV5BMjI0NT...,The world famous singing pre-teen chipmunk tri...,Alvin and the Chipmunks: The Squeakquel,3223,5280,"[Jason Lee, Zachary Levi, David Cross]",2009,tt1231580,add,[The world famous singing pre-teen chipmunk tr...
3,movies,tt0804452,None,[2.7],[Sean McNamara],2007-08-03T00:00:00Z,2.7,"[Comedy, Family, Music]",https://m.media-amazon.com/images/M/MV5BMTUxND...,"During their first year of high school, four b...",Bratz,3064,6600,"[Skyler Shaye, Janel Parrish, Logan Browning]",2007,tt0804452,add,[During their first year of high <em>school</e...


このように最もシンプルにページングを実装できる from + to ですが、以下の懸念事項があります

- 検索は Search API の都度新規に実行されるため、処理に一貫性はありません。前回の Search API の実行からデータの変化がある場合、paging の結果にも影響します
- 検索は from と size の範囲に関係なく実行されるため、結果を分割取得したとしても、Search API を発行する都度、毎回同じ検索負荷が生じます
- デフォルトでは、from + size は 10000 を超えることができません。`index.max_result_window` の[インデックス設定][index-settings]を変更することでこの制限を緩和できますが、パフォーマンスへの影響があるため一般的には推奨されません。

数万件の結果を取得し paging を行う場合は、異なる手法を検討する必要があります。

[index-settings]: https://opensearch.org/docs/latest/install-and-configure/configuring-opensearch/index-settings/

In [56]:
index_name = "movies"

payload = {
  "from": 9999,
  "size": 2,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

try:
    response = opensearch_client.search(
        index=index_name,
        body=payload
    )
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)


RequestError(400, 'search_phase_execution_exception', 'Result window is too large, from + size must be less than or equal to: [10000] but was [10001]. See the scroll api for a more efficient way to request large data sets. This limit can be set by changing the [index.max_result_window] index level setting.')


#### scroll によるページング
scroll は機械学習ジョブといった PB （ペタバイト）クラスのデータを取得するバッチ処理に向いている手法です。リクエストヘッダに scroll パラメーターを付与することで有効化されます。scroll はリクエスト時点のスナップショットを取得するため、メモリを大量に消費する場合があります。この特性上、頻繁に実行されるクエリには向いていません。

scroll オプションを使用して search クエリを実行すると、実行結果に "_scroll_id" が付与されます。_scroll_id を次回のリクエストに付与することで、後続の結果を得ることができます。

In [57]:
index_name = "movies"

payload = {
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "school"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload,
    scroll="10m"
)

scroll_id = response["_scroll_id"]
print("ScrollId: " + scroll_id)
pd.json_normalize(response["hits"]["hits"])


ScrollId: FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoCBZack5KSmJvRlJXS3RvT0ZfTU5PYUl3AAAAAAAAA2wMMTAuMC4xNjUuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADbwwxMC4wLjE2NS4xMjgWeDN0Z3FIbFZTeEtxeG9Uenc1OVZCUQAAAAAAAAJ1DDEwLjAuMTIzLjEyOBZ4M3RncUhsVlN4S3F4b1R6dzU5VkJRAAAAAAAAAnYMMTAuMC4xMjMuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADbgwxMC4wLjE2NS4xMjgWWnJOSkpib0ZSV0t0b09GX01OT2FJdwAAAAAAAANtDDEwLjAuMTY1LjEyOBZ4M3RncUhsVlN4S3F4b1R6dzU5VkJRAAAAAAAAAncMMTAuMC4xMjMuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADcAwxMC4wLjE2NS4xMjg=


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0088763,None,[8.5],[Robert Zemeckis],1985-07-03T00:00:00Z,8.5,"[Adventure, Comedy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTk4OT...,A teenager is accidentally sent 30 years into ...,Back to the Future,204,6960,"[Michael J. Fox, Christopher Lloyd, Lea Thompson]",1985,tt0088763,add
1,movies,tt0181875,None,[7.9],[Cameron Crowe],2000-09-08T00:00:00Z,7.9,"[Drama, Music]",https://m.media-amazon.com/images/M/MV5BMTI0MD...,A high-school boy is given the chance to write...,Almost Famous,599,7320,"[Billy Crudup, Frances McDormand, Kate Hudson]",2000,tt0181875,add
2,movies,tt0088847,None,[7.9],[John Hughes],1985-02-07T00:00:00Z,7.9,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BMzYyNT...,"Five high school students, all different stere...",The Breakfast Club,120,5820,"[Emilio Estevez, Judd Nelson, Molly Ringwald]",1985,tt0088847,add
3,movies,tt0091042,None,[7.800000000000001],[John Hughes],1986-06-11T00:00:00Z,7.8,"[Comedy, Drama]",https://m.media-amazon.com/images/M/MV5BMTg2MT...,A high school wise guy is determined to have a...,Ferris Bueller's Day Off,528,6180,"[Matthew Broderick, Alan Ruck, Mia Sara]",1986,tt0091042,add
4,movies,tt1250777,None,[7.800000000000001],[Matthew Vaughn],2010-03-12T00:00:00Z,7.8,"[Action, Comedy]",https://m.media-amazon.com/images/M/MV5BMTMzNz...,Dave Lizewski is an unnoticed high school stud...,Kick-Ass,162,7020,"[Aaron Taylor-Johnson, Nicolas Cage, Chloë Gra...",2010,tt1250777,add


In [58]:
payload = {
  scroll_id: scroll_id,
  "size": 5,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": {
              "query": "school"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    scroll="10m",
)

scroll_id = response["_scroll_id"]
print("ScrollId: " + scroll_id)
pd.json_normalize(response["hits"]["hits"])

ScrollId: FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoCBZack5KSmJvRlJXS3RvT0ZfTU5PYUl3AAAAAAAAA3MMMTAuMC4xNjUuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADdQwxMC4wLjE2NS4xMjgWWnJOSkpib0ZSV0t0b09GX01OT2FJdwAAAAAAAANyDDEwLjAuMTY1LjEyOBZack5KSmJvRlJXS3RvT0ZfTU5PYUl3AAAAAAAAA3EMMTAuMC4xNjUuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADdwwxMC4wLjE2NS4xMjgWeDN0Z3FIbFZTeEtxeG9Uenc1OVZCUQAAAAAAAAJ4DDEwLjAuMTIzLjEyOBZack5KSmJvRlJXS3RvT0ZfTU5PYUl3AAAAAAAAA3QMMTAuMC4xNjUuMTI4FlpyTkpKYm9GUldLdG9PRl9NTk9hSXcAAAAAAAADdgwxMC4wLjE2NS4xMjg=


,_index,_id,_score,_source.directors,_source.release_date,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,_source.rating
0,movies,tt1951264,1.0,[Francis Lawrence],2013-11-11T00:00:00Z,"[Action, Adventure, Sci-Fi, Thriller]",https://m.media-amazon.com/images/M/MV5BMTAyMj...,Katniss Everdeen and Peeta Mellark become targ...,The Hunger Games: Catching Fire,4,8760.0,"[Jennifer Lawrence, Josh Hutcherson, Liam Hems...",2013,tt1951264,add,NaN
1,movies,tt2109248,1.0,[Michael Bay],2014-06-25T00:00:00Z,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTQyMD...,A mechanic and his daughter make a discovery t...,Transformers: Age of Extinction,10,NaN,"[Mark Wahlberg, Nicola Peltz, Jack Reynor]",2014,tt2109248,add,NaN
2,movies,tt1670345,1.0,[Louis Leterrier],2013-05-21T00:00:00Z,"[Crime, Mystery, Thriller]",https://m.media-amazon.com/images/M/MV5BMTY0ND...,An FBI agent and an Interpol detective track a...,Now You See Me,11,6900.0,"[Jesse Eisenberg, Common, Mark Ruffalo]",2013,tt1670345,add,7.3
3,movies,tt1815862,1.0,[M. Night Shyamalan],2013-05-01T00:00:00Z,"[Action, Adventure, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BMTY3Mz...,A crash landing leaves Kitai Raige and his fat...,After Earth,17,6000.0,"[Jaden Smith, David Denman, Will Smith]",2013,tt1815862,add,4.9
4,movies,tt1234721,1.0,[José Padilha],2014-01-30T00:00:00Z,"[Action, Crime, Sci-Fi, Thriller]",https://m.media-amazon.com/images/M/MV5BMjMwNz...,"In 2028 Detroit, when Alex Murphy (Joel Kinnam...",RoboCop,30,NaN,"[Joel Kinnaman, Douglas Urbanski, Abbie Cornish]",2014,tt1234721,add,NaN
5,movies,tt2395427,1.0,[Joss Whedon],2015-04-29T00:00:00Z,"[Action, Adventure, Fantasy, Sci-Fi]",https://m.media-amazon.com/images/M/MV5BNjA3MT...,NaN,The Avengers: Age of Ultron,40,NaN,"[Scarlett Johansson, Chris Hemsworth, James Sp...",2015,tt2395427,add,NaN
6,movies,tt2024544,1.0,[Steve McQueen],2013-08-30T00:00:00Z,"[Biography, Drama, History]",https://m.media-amazon.com/images/M/MV5BMjExMT...,"In the antebellum United States, Solomon North...",12 Years a Slave,44,7980.0,"[Chiwetel Ejiofor, Michael K. Williams, Michae...",2013,tt2024544,add,7.7
7,movies,tt1981677,1.0,[Jason Moore],2012-09-28T00:00:00Z,"[Comedy, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTcyMT...,"Beca, a freshman at Barden University, is cajo...",Pitch Perfect,46,6720.0,"[Anna Kendrick, Brittany Snow, Rebel Wilson]",2012,tt1981677,add,7.1
8,movies,tt1758595,1.0,[Oliver Hirschbiegel],2013-09-05T00:00:00Z,"[Biography, Drama, Romance]",https://m.media-amazon.com/images/M/MV5BMjA0ND...,"During the last two years of her life, Princes...",Diana,54,6780.0,"[Naomi Watts, Naveen Andrews, Douglas Hodge]",2013,tt1758595,add,4.6
9,movies,tt1690953,1.0,"[Pierre Coffin, Chris Renaud]",2013-06-16T00:00:00Z,"[Animation, Adventure, Comedy, Crime, Family, ...",https://m.media-amazon.com/images/M/MV5BOTg4NT...,Gru is recruited by the Anti-Villain League to...,Despicable Me 2,55,5880.0,"[Steve Carell, Kristen Wiig, Benjamin Bratt]",2013,tt1690953,add,7.7


#### search_after によるページング
[Search after][search_after] は from + size に似たアプローチですが、前回取得した範囲は検索結果から除外できるメリットがあります。検索結果を全件取得してから部分的に切り出しを行う from + size と比較して、ページングによるオーバーヘッドを抑えられます。

以下は Search after のサンプルです。全部で 8 件の検索結果を、4 件ずつに分けて取得しています。

1 つ目のクエリでは rating フィールドで降順ソートされた結果から size = 4 で先頭 4 件を取得しています。2 つ目のクエリでは、search_after オプションに 1 つ目のクエリ実行結果の 4 件目の ratings の値を入れています。これにより、1 つ目のクエリで取得した結果より後の結果から 4 件のドキュメントを取得することに成功しています。

[search_after]: https://opensearch.org/docs/latest/search-plugins/searching-data/paginate/#the-search_after-parameter

In [59]:
index_name = "movies"
payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

search_after_rating = response["hits"]["hits"][-1]["sort"]
print("search_after_rating: " + str(search_after_rating))
pd.json_normalize(response["hits"]["hits"])

search_after_rating: [5.800000000000001]


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0332379,None,[7.0],[Richard Linklater],2003-09-09T00:00:00Z,7.0,"[Comedy, Music]",https://m.media-amazon.com/images/M/MV5BMjEwOT...,A wannabe rock star in need of cash poses as a...,The School of Rock,727,6480,"[Jack Black, Mike White, Joan Cusack]",2003,tt0332379,add,[A wannabe rock star in need of cash poses as ...
1,movies,tt0981227,None,[6.7],[Peter Sollett],2008-09-06T00:00:00Z,6.7,"[Comedy, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BOTY2Mj...,"High school student Nick O'Leary, member of th...",Nick and Norah's Infinite Playlist,1701,5400,"[Michael Cera, Kat Dennings, Aaron Yoo]",2008,tt0981227,add,"[High <em>school</em> student Nick O'Leary, me..."
2,movies,tt0462590,None,[6.2],[Anne Fletcher],2006-08-07T00:00:00Z,6.2,"[Crime, Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTIxMD...,Tyler Gage receives the opportunity of a lifet...,Step Up,947,6240,"[Channing Tatum, Jenna Dewan-Tatum, Damaine Ra...",2006,tt0462590,add,[Tyler Gage receives the opportunity of a life...
3,movies,tt1023481,None,[5.800000000000001],[Jon M. Chu],2008-02-14T00:00:00Z,5.8,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTc0Nz...,Romantic sparks occur between two dance studen...,Step Up 2: The Streets,1427,5880,"[Robert Hoffman, Briana Evigan, Cassie Ventura]",2008,tt1023481,add,[Romantic sparks occur between two dance stude...


In [60]:
index_name = "movies"

payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ],
      "should": [
        {
          "term": {
            "genres": {
              "value": "Romance"
            }
          }
        },
        {
          "term": {
            "genres": {
              "value": "Comedy"
            }
          }
        }
      ],
      "minimum_should_match": 1,
      "filter": [
        {
          "range": {
            "year": {
              "gte": 2000,
              "lte": 2009
            }
          }
        }
      ],
      "must_not": [
        {
          "terms": {
            "genres": ["Thriller", "Horror"]
          }
        }
      ]
    }
  },
  "highlight": {
    "fields": {
      "plot": {}
    }
  },
  "sort": [
    {
       "rating": {
          "order": "desc"
      }
    }
  ],
  "search_after": search_after_rating
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

search_after_rating = response["hits"]["hits"][-1]["sort"]
print("search_after_rating: " + str(search_after_rating))
pd.json_normalize(response["hits"]["hits"])

search_after_rating: [2.7]


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type,highlight.plot
0,movies,tt0361696,None,[5.5],[Sean McNamara],2004-10-03T00:00:00Z,5.5,"[Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTM1MT...,A girl from a small town heads to the big city...,Raise Your Voice,3425,6180,"[Hilary Duff, John Corbett, Rebecca De Mornay]",2004,tt0361696,add,[from a small town heads to the big city of Lo...
1,movies,tt0306841,None,[4.9],[Jim Fall],2003-04-26T00:00:00Z,4.9,"[Adventure, Comedy, Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTMzND...,Lizzie McGuire has graduated from middle schoo...,The Lizzie McGuire Movie,3219,5640,"[Hilary Duff, Adam Lamberg, Clayton Snyder]",2003,tt0306841,add,[Lizzie McGuire has graduated from middle <em>...
2,movies,tt1231580,None,[4.2],[Betty Thomas],2009-12-11T00:00:00Z,4.2,"[Animation, Comedy, Family, Fantasy, Music]",https://m.media-amazon.com/images/M/MV5BMjI0NT...,The world famous singing pre-teen chipmunk tri...,Alvin and the Chipmunks: The Squeakquel,3223,5280,"[Jason Lee, Zachary Levi, David Cross]",2009,tt1231580,add,[The world famous singing pre-teen chipmunk tr...
3,movies,tt0804452,None,[2.7],[Sean McNamara],2007-08-03T00:00:00Z,2.7,"[Comedy, Family, Music]",https://m.media-amazon.com/images/M/MV5BMTUxND...,"During their first year of high school, four b...",Bratz,3064,6600,"[Skyler Shaye, Janel Parrish, Logan Browning]",2007,tt0804452,add,[During their first year of high <em>school</e...


<div class="alert alert-block alert-info"> 
<b>Tips: sort 条件を明示的に指定しない場合に Search after を利用する方法</b>

デフォルトでは OpenSearch はスコア順にドキュメントのソートを行います。明示的にソート対象のフィールドに "_score" を指定した場合と挙動は同じです。この特性を利用して、sort に _score による降順ソートの条件を書くことでスコアによるソートと Search after を両立できます。ただし、ソート条件が _score だけだとエラーになってしまうため、id など何かしらのフィールドとセットで sort 条件を書く必要があります。

</div>

In [61]:
index_name = "movies"

payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
       "_score": {
          "order": "desc"
      }
    },
    {
        "id": {
            "order": "asc"
        }
    }
  ]
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

search_after_score = response["hits"]["hits"][-1]["sort"]
print("search_after_score: " + str(search_after_score))
pd.json_normalize(response["hits"]["hits"])

search_after_score: [8.190945, 'tt0080716']


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0332379,8.694975,"[8.694975, tt0332379]",[Richard Linklater],2003-09-09T00:00:00Z,7.0,"[Comedy, Music]",https://m.media-amazon.com/images/M/MV5BMjEwOT...,A wannabe rock star in need of cash poses as a...,The School of Rock,727,6480,"[Jack Black, Mike White, Joan Cusack]",2003,tt0332379,add
1,movies,tt0113862,8.490513,"[8.490513, tt0113862]",[Stephen Herek],1995-12-29T00:00:00Z,7.1,"[Drama, Music]",https://m.media-amazon.com/images/M/MV5BMTUxOD...,A frustrated composer finds fulfillment as a h...,Mr. Holland's Opus,1841,8580,"[Richard Dreyfuss, Glenne Headly, Jay Thomas]",1995,tt0113862,add
2,movies,tt0361696,8.390359,"[8.390359, tt0361696]",[Sean McNamara],2004-10-03T00:00:00Z,5.5,"[Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTM1MT...,A girl from a small town heads to the big city...,Raise Your Voice,3425,6180,"[Hilary Duff, John Corbett, Rebecca De Mornay]",2004,tt0361696,add
3,movies,tt0080716,8.190945,"[8.190945, tt0080716]",[Alan Parker],1980-05-16T00:00:00Z,6.4,"[Drama, Music]",https://m.media-amazon.com/images/M/MV5BMjAyOT...,A chronicle of the lives of several teenagers ...,Fame,3755,8040,"[Eddie Barth, Irene Cara, Lee Curreri]",1980,tt0080716,add


In [62]:
index_name = "movies"

payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
      "_score": {
          "order": "desc"
      },
      "id": {
          "order": "asc"
      }
    }
  ],
  "search_after": search_after_score
}

response = opensearch_client.search(
    index=index_name,
    body=payload
)

search_after_score = response["hits"]["hits"][-1]["sort"]
print("search_after_score: " + str(search_after_score))
pd.json_normalize(response["hits"]["hits"])

search_after_score: [7.837826, 'tt1231580']


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1023481,7.992006,"[7.992006, tt1023481]",[Jon M. Chu],2008-02-14T00:00:00Z,5.8,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTc0Nz...,Romantic sparks occur between two dance studen...,Step Up 2: The Streets,1427,5880,"[Robert Hoffman, Briana Evigan, Cassie Ventura]",2008,tt1023481,add
1,movies,tt1447972,7.883687,"[7.8836875, tt1447972]","[Max Giwa, Dania Pasquini]",2010-05-19T00:00:00Z,5.6,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMzc4MD...,In order to win the Street Dance Championships...,StreetDance 3D,1210,5880,"[Nichola Burley, Richard Winsor, Ukweli Roach]",2010,tt1447972,add
2,movies,tt0085549,7.858109,"[7.858109, tt0085549]",[Adrian Lyne],1983-04-15T00:00:00Z,5.8,"[Drama, Romance, Music]",https://m.media-amazon.com/images/M/MV5BMjA5Nj...,A Pittsburgh woman with two jobs as a welder a...,Flashdance,1423,5700,"[Jennifer Beals, Michael Nouri, Lilia Skala]",1983,tt0085549,add
3,movies,tt1231580,7.837826,"[7.837826, tt1231580]",[Betty Thomas],2009-12-11T00:00:00Z,4.2,"[Animation, Comedy, Family, Fantasy, Music]",https://m.media-amazon.com/images/M/MV5BMjI0NT...,The world famous singing pre-teen chipmunk tri...,Alvin and the Chipmunks: The Squeakquel,3223,5280,"[Jason Lee, Zachary Levi, David Cross]",2009,tt1231580,add


#### Search after + PIT(Point in Time) によるページング
[PIT(Point in Time)][point-in-time] は、インデックスのある時点の固定された状態を作り出す機能です。PIT と Search after を組み合わせることで一貫性のあるページングされた結果を取得することができます。

PIT を使用した検索の流れは以下の通りです。PIT 作成時に keep_alive パラメーターで PIT の保持時間を指定するため、PIT の削除は任意です。

1. PIT の作成
2. PIT を使用した検索の実行
3. (option) PIT の削除

PIT を使用する場合、PIT 側に Index の情報が入っているため、Search API 実行時にインデックス名の指定は行いません。リクエストパラメーターにインデックス名を含めてはいけない点に注意が必要です。

[point-in-time]: https://opensearch.org/docs/latest/search-plugins/searching-data/point-in-time/

In [63]:
response = opensearch_client.create_point_in_time(
    index=index_name,
    keep_alive="10m"
)

pit_id = response.get("pit_id")
print('\n Point in time ID: '+ pit_id)


 Point in time ID: ExYifP7AQGiwTywQwHXCWQ


In [64]:
index_name = "movies"

payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
      "_score": {
          "order": "desc"
      },
      "id": {
          "order": "asc"
      }
    }
  ],
  "pit": {
    "id":  pit_id, 
    "keep_alive": "10m"
  },
}

response = opensearch_client.search(
    body=payload
)

search_after_score = response["hits"]["hits"][-1]["sort"]
print("search_after_score: " + str(search_after_score))
pd.json_normalize(response["hits"]["hits"])

search_after_score: [8.190945, 'tt0080716']


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt0332379,8.694975,"[8.694975, tt0332379]",[Richard Linklater],2003-09-09T00:00:00Z,7.0,"[Comedy, Music]",https://m.media-amazon.com/images/M/MV5BMjEwOT...,A wannabe rock star in need of cash poses as a...,The School of Rock,727,6480,"[Jack Black, Mike White, Joan Cusack]",2003,tt0332379,add
1,movies,tt0113862,8.490513,"[8.490513, tt0113862]",[Stephen Herek],1995-12-29T00:00:00Z,7.1,"[Drama, Music]",https://m.media-amazon.com/images/M/MV5BMTUxOD...,A frustrated composer finds fulfillment as a h...,Mr. Holland's Opus,1841,8580,"[Richard Dreyfuss, Glenne Headly, Jay Thomas]",1995,tt0113862,add
2,movies,tt0361696,8.390359,"[8.390359, tt0361696]",[Sean McNamara],2004-10-03T00:00:00Z,5.5,"[Family, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTM1MT...,A girl from a small town heads to the big city...,Raise Your Voice,3425,6180,"[Hilary Duff, John Corbett, Rebecca De Mornay]",2004,tt0361696,add
3,movies,tt0080716,8.190945,"[8.190945, tt0080716]",[Alan Parker],1980-05-16T00:00:00Z,6.4,"[Drama, Music]",https://m.media-amazon.com/images/M/MV5BMjAyOT...,A chronicle of the lives of several teenagers ...,Fame,3755,8040,"[Eddie Barth, Irene Cara, Lee Curreri]",1980,tt0080716,add


In [65]:
index_name = "movies"

payload = {
  "size": 4,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "plot": "school"
          }
        },
        {
          "term": {
            "genres": {
              "value": "Music"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
      "_score": {
          "order": "desc"
      },
      "id": {
          "order": "asc"
      }
    }
  ],
  "pit": {
    "id":  pit_id, 
    "keep_alive": "10m"
  },
  "search_after": search_after_score
}

response = opensearch_client.search(
    body=payload
)

search_after_score = response["hits"]["hits"][-1]["sort"]
print("search_after_score: " + str(search_after_score))
pd.json_normalize(response["hits"]["hits"])

search_after_score: [7.837826, 'tt1231580']


,_index,_id,_score,sort,_source.directors,_source.release_date,_source.rating,_source.genres,_source.image_url,_source.plot,_source.title,_source.rank,_source.running_time_secs,_source.actors,_source.year,_source.id,_source.type
0,movies,tt1023481,7.992006,"[7.992006, tt1023481]",[Jon M. Chu],2008-02-14T00:00:00Z,5.8,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMTc0Nz...,Romantic sparks occur between two dance studen...,Step Up 2: The Streets,1427,5880,"[Robert Hoffman, Briana Evigan, Cassie Ventura]",2008,tt1023481,add
1,movies,tt1447972,7.883687,"[7.8836875, tt1447972]","[Max Giwa, Dania Pasquini]",2010-05-19T00:00:00Z,5.6,"[Drama, Music, Romance]",https://m.media-amazon.com/images/M/MV5BMzc4MD...,In order to win the Street Dance Championships...,StreetDance 3D,1210,5880,"[Nichola Burley, Richard Winsor, Ukweli Roach]",2010,tt1447972,add
2,movies,tt0085549,7.858109,"[7.858109, tt0085549]",[Adrian Lyne],1983-04-15T00:00:00Z,5.8,"[Drama, Romance, Music]",https://m.media-amazon.com/images/M/MV5BMjA5Nj...,A Pittsburgh woman with two jobs as a welder a...,Flashdance,1423,5700,"[Jennifer Beals, Michael Nouri, Lilia Skala]",1983,tt0085549,add
3,movies,tt1231580,7.837826,"[7.837826, tt1231580]",[Betty Thomas],2009-12-11T00:00:00Z,4.2,"[Animation, Comedy, Family, Fantasy, Music]",https://m.media-amazon.com/images/M/MV5BMjI0NT...,The world famous singing pre-teen chipmunk tri...,Alvin and the Chipmunks: The Squeakquel,3223,5280,"[Jason Lee, Zachary Levi, David Cross]",2009,tt1231580,add


### 非同期検索
大規模なデータ検索、特にウォームノードや複数のリモートクラスターにまたがって検索が実行される場合、完了までに時間がかかることがあります。
完了までクライアントが待機するにはタイムアウトを延長するなどの措置が必要ですが、何らかの問題で接続が切断された場合は改めて検索リクエストを発行する必要があります。

このような課題に対処するために、OpenSearch の[非同期検索][async]を使用することができます。非同期検索は、バックグラウンドで実行される検索リクエストを送信できます。検索の進行状況は監視可能であり、結果を段階的に取得することも可能です。検索結果は任意の期間保存することが可能であり、後から取得することもできます。

[async]: https://opensearch.org/docs/latest/search-plugins/async/index/

> AOSS では非同期検索はできません。

In [67]:
# index_name = "movies"

# payload = {
#   "index": index_name
# }

# response = opensearch_client.http.post(
#     url = "/_plugins/_asynchronous_search?index=" + index_name
# )

# asynchronous_search_id = response["id"]
# print("asynchronous_search_id: " + asynchronous_search_id)
# pd.json_normalize(response["response"]["hits"]["hits"])

In [70]:
# index_name = "movies"

# payload = {
#   "index": index_name
# }

# response = opensearch_client.http.get(
#     url = "/_plugins/_asynchronous_search/stats"
# )

# print(json.dumps(response, indent=2))

### 検索と組み合わせたドキュメント処理
OpenSearch では、query で抽出したドキュメントの一括更新や削除を行うことが可能です。以降のセクションで、具体的な実行方法を確認していきます。

#### 複数ドキュメントの一括更新
[Update by query][update-by-query] を使用することで、特定条件に一致する複数ドキュメントのデータ更新をまとめて行うことが可能です。条件は Search API と同様に query で指定します。

以下のサンプルでは、year が 1920 から 1923 までの movies 内のドキュメントについて、rating の値を 0.1 増加させています。レスポンスの total フィールドに、処理対象となったドキュメントの件数が記載されています。

[update-by-query]: https://opensearch.org/docs/latest/api-reference/document-apis/update-by-query/

> AOSS では、`update_by_query()` はできません。

In [72]:
# index_name = "movies"

# payload = {
#   "query": {
#     "range": {
#       "year": {
#         "gte": 1920,
#         "lte": 1923
#       }
#     }
#   },
#   "script" : {
#     "source": "ctx._source.rating += params.delta",
#     "lang": "painless",
#     "params" : {
#       "delta" : 0.1
#     }
#   }
# }

# response = opensearch_client.update_by_query(
#     index = index_name,
#     body = payload,
#     refresh = True
# )

# print(json.dumps(response, indent=2))

#### 複数ドキュメントの一括削除
[Delete by query][delete-by-query] は、特定条件に一致する複数ドキュメントを一括で削除します。条件は Search API と同様に query で指定します。

以下のサンプルでは、year が 1989 までの movies 内のドキュメントを削除します。レスポンスの total フィールドに、処理対象となったドキュメントの件数が記載されています。

[delete-by-query]: https://opensearch.org/docs/latest/api-reference/document-apis/delete-by-query/

> AOSS では、`delete_by_query()` はできません。

In [74]:
# index_name = "movies"

# payload = {
#   "query": {
#     "range": {
#       "year": {
#         "lte": 1989
#       }
#     }
#   }
# }

# response = opensearch_client.delete_by_query(
#     index = index_name,
#     body = payload,
#     refresh = True
# )

# print(json.dumps(response, indent=2))

## まとめ
本ラボでは、OpenSearch の基本的な検索クエリについて、ユースケースと実際の使い方を解説しました。本ラボで学習した内容を元に、次のステップとして以下のラボを実行してみましょう。

- [日本語全文検索の実装](3-full-text-search-jp.ipynb)
- [ベクトル検索の実装 (Bedrock 編)](4-ai-search.ipynb)

## 後片付け

### インデックス削除
本ワークショップで使用したインデックスを削除します。インデックスの削除は Delete index API で行います。インデックスを削除するとインデックス内のドキュメントも削除されます。

In [75]:
index_name = "movies"

try:
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

{
  "acknowledged": true
}


### データセット削除
ダウンロードしたデータセットを削除します。./dataset ディレクトリ配下に何もない場合は、./dataset ディレクトリも合わせて削除します。

In [76]:
%rm -rf {dataset_dir}

In [77]:
%rmdir ./dataset